In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from drive.MyDrive.graduation_project.utils.utils import get_patients,fitted_curve,to_polynomial
from sklearn.model_selection import train_test_split,KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [85]:
data = pd.read_csv('/content/drive/MyDrive/graduation_project/data.csv')

In [86]:
data.head()

,Patient,Weeks,FVC,Percent,Sex,Ex-smoker,Never-smoked,Currently-smokes,decade
0,ID00007637202177411956430,-4,2315,58.253649,0,1,0,0,7
1,ID00007637202177411956430,1,2251,58.253649,0,1,0,0,7
2,ID00007637202177411956430,5,2214,58.253649,0,1,0,0,7
3,ID00007637202177411956430,7,2061,58.253649,0,1,0,0,7
4,ID00007637202177411956430,9,2144,58.253649,0,1,0,0,7


In [87]:
image_features = pd.read_csv('/content/drive/MyDrive/graduation_project/Copy of final_features.csv')

In [88]:
image_features.head()

,Patient,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,ID00007637202177411956430,380.149350,-206.351027,-156.433460,33.208309,62.585335,-286.028391,23.054035,173.602477,15.039002,-32.232864,108.707270,-159.291377,52.448239,-144.839725,73.091950,-92.603068,184.283319,-31.817387,-223.343122,48.986629,78.685981,191.682613,39.453683,-161.363610,-151.476868,71.588827,17.946780,-212.503244,174.745495,43.672575,-108.384653,63.161517
1,ID00009637202177434476278,-3493.767021,235.598640,-239.190103,130.059535,-200.854582,139.390113,-172.697327,59.148479,-53.915173,-89.773163,-1.377660,92.443528,13.874730,15.405616,-22.799429,-66.829909,-68.530495,-21.174953,-136.182374,-72.844752,15.415771,-85.762851,142.416749,-50.805719,-87.596642,18.585889,79.909472,-23.024210,-15.424642,-113.465564,2.390164,-50.468937
2,ID00010637202177584971671,-686.991998,-233.966341,-111.472365,-130.170477,-537.116039,-217.162957,-251.206076,177.654474,-108.840526,-145.027600,95.838591,11.154422,-147.068815,122.209971,55.160645,-16.152181,-136.742642,288.196746,-27.433825,64.278432,105.029141,-114.629113,-23.653840,41.365189,-3.472775,-61.276703,108.466273,-30.935653,-81.154129,-105.855858,-172.318492,152.708027
3,ID00011637202177653955184,8079.404722,-554.232637,-17.162803,74.473691,202.660579,-127.668875,258.994581,30.598155,92.303766,71.166998,94.417285,-91.654179,-80.982290,71.120235,-22.815301,-79.600455,47.550929,-0.724792,32.624263,53.185526,-48.197889,1.904115,6.560237,33.273568,-18.327816,14.669127,17.685314,37.247338,-51.366246,-9.860449,-26.796445,11.339162
4,ID00012637202177665765362,-2781.144865,-152.011328,308.380157,-132.000223,-93.401310,183.994670,-119.753805,-179.779238,103.060708,38.157641,36.375796,-59.655949,-21.147534,49.405072,-82.532633,-169.012898,127.218860,18.236679,-113.792255,225.534603,-16.268478,53.445677,44.758343,35.381085,27.012688,70.794659,-43.635250,21.055317,46.673699,53.161754,-78.785774,52.218642


In [89]:
new_features = pd.DataFrame(columns=image_features.columns)

In [90]:
for i in range(len(image_features)):
  for _ in range(10):
    new_features.loc[len(new_features)] = image_features.iloc[i]

In [69]:
final = pd.concat([data,new_features.drop('Patient',axis=1)],axis=1)

In [91]:
final.head()

,Patient,Weeks,FVC,Percent,Sex,Ex-smoker,Never-smoked,Currently-smokes,decade,0,1,2,3,4,5,6,7,8,9
0,ID00007637202177411956430,-4,2315,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514
1,ID00007637202177411956430,1,2251,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514
2,ID00007637202177411956430,5,2214,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514
3,ID00007637202177411956430,7,2061,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514
4,ID00007637202177411956430,9,2144,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514


In [92]:
final.to_csv('/content/drive/MyDrive/graduation_project/final_data2.csv',index=False)

In [93]:
data = final

In [94]:
data.head()

,Patient,Weeks,FVC,Percent,Sex,Ex-smoker,Never-smoked,Currently-smokes,decade,0,1,2,3,4,5,6,7,8,9
0,ID00007637202177411956430,-4,2315,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514
1,ID00007637202177411956430,1,2251,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514
2,ID00007637202177411956430,5,2214,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514
3,ID00007637202177411956430,7,2061,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514
4,ID00007637202177411956430,9,2144,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514


---

FVCt = base_FVC + (t - first_t) * (W1 * X)

---


In [95]:
def Base(data):
  return data[data.index[0]]

In [96]:
groubed = data.groupby('Patient')[['Weeks','FVC']].agg(Base)

In [97]:
groubed['Base_Week'] = groubed['Weeks']
groubed['Base_FVC'] = groubed['FVC']

In [98]:
groubed.drop(['Weeks','FVC'],axis=1,inplace=True)

In [99]:
data = data.join(groubed,'Patient')

In [100]:
data.head()

,Patient,Weeks,FVC,Percent,Sex,Ex-smoker,Never-smoked,Currently-smokes,decade,0,1,2,3,4,5,6,7,8,9,Base_Week,Base_FVC
0,ID00007637202177411956430,-4,2315,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514,-4,2315
1,ID00007637202177411956430,1,2251,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514,-4,2315
2,ID00007637202177411956430,5,2214,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514,-4,2315
3,ID00007637202177411956430,7,2061,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514,-4,2315
4,ID00007637202177411956430,9,2144,58.253649,0,1,0,0,7,380.14935,-206.351027,-156.43346,33.208315,62.585326,-286.028529,23.054767,173.611203,15.036581,-32.231514,-4,2315


In [82]:
def mae(target,predicted):
   return torch.mean(torch.abs(target - predicted))

In [83]:
class Model:

  def __init__(self,degree = 1,device='cpu',loss=mae):
    self.degree = degree
    self.W = torch.randn((7+32) * degree , requires_grad=True).to(device)
    self.history = ([],[])
    self.log = ([],[])
    self.device = device
    self.loss = loss
  
  def get_data(self,data):

    intersect = np.array(data['Base_FVC'])
    intersect = torch.from_numpy(intersect.astype(np.float32)).to(self.device)

    base_week = np.array(data['Base_Week'])
    base_week = torch.from_numpy(base_week.astype(np.float32)).to(self.device)

    current_week = np.array(data['Weeks'])
    current_week = torch.from_numpy(current_week.astype(np.float32)).to(self.device)


    target_value = np.array(data['FVC'])
    target_value = torch.from_numpy(target_value.astype(np.float32)).to(self.device)

    features = np.array(data.drop(['Patient','FVC','Base_Week','Base_FVC'],axis=1))
    features = to_polynomial(features.T,self.degree,False)
    assert features.shape[1] == (7+32) * self.degree
    features = torch.from_numpy(features.astype(np.float32)).to(self.device)
    return intersect, base_week, current_week, features, target_value

  def predict(self,intersect, base_week, current_week, features):
      return intersect + (current_week - base_week) * (features @ self.W)


  def fit(self,data,alpha = 0.000001,n_epoch = 100,_print = False,print_after=1):
    
    train_data , val_data = train_test_split(data,test_size=0.25,random_state = 40,shuffle=False)
    intersect_train, base_week_train, current_week_train, features_train, target_value_train = self.get_data(train_data)
    intersect_val, base_week_val, current_week_val, features_val, target_value_val = self.get_data(val_data)
    lr = alpha

    for epoch in range(n_epoch):
      decay_rate  = 0.00001
      predicted_value_train = self.predict(intersect_train, base_week_train, current_week_train, features_train)      
      train_loss = self.loss(target_value_train,predicted_value_train)
      predicted_value_val = self.predict(intersect_val, base_week_val, current_week_val, features_val)
      val_loss = self.loss(target_value_val,predicted_value_val)
      
      if epoch % print_after == 0:
        if _print:
          print(f'epoch > {epoch}',f'train loss = {train_loss.item()}',f'val loss = {val_loss.item()}',f'lr = {lr}')
        self.log[0].append(train_loss.item())
        self.log[1].append(val_loss.item())

      self.history[0].append(train_loss.item())
      self.history[1].append(val_loss.item())
      train_loss.backward()
      with torch.no_grad():
        self.W -= lr * self.W.grad
        self.W.grad.zero_()

      # lr decay
      if epoch >=6:   
        mean_loss = np.mean(self.history[1][-6:-1])
        if val_loss.item() > mean_loss:
          lr = max((1/(1 + decay_rate * epoch)) * alpha, 1e-15)
          

In [22]:
root_path = '/content/drive/MyDrive/graduation_project/summary/'

In [84]:
train,test = train_test_split(data,test_size=0.25,random_state = 40,shuffle=False)
model = Model(1,device='cpu',loss=mae)
start_time = time.perf_counter()
model.fit(train,alpha=5e-6,n_epoch=int(2e6),print_after=10000,_print=True)
end_time = time.perf_counter()
intersect, base_week, current_week, features, target_value = model.get_data(test)
p = model.predict(intersect,base_week,current_week,features)
print(f'model->',mae(target_value,p).item(),'time(sec) -> ',end_time-start_time)

epoch > 0 train loss = 16864.537109375 val loss = 15608.1943359375 lr = 5e-06
epoch > 10000 train loss = 1753.4571533203125 val loss = 1609.3363037109375 lr = 4.545495868144256e-06
epoch > 20000 train loss = 1700.380615234375 val loss = 1584.265869140625 lr = 4.166701389178243e-06
epoch > 30000 train loss = 1545.2965087890625 val loss = 1439.843017578125 lr = 3.8461834321802475e-06
epoch > 40000 train loss = 1362.2666015625 val loss = 1261.829345703125 lr = 3.57145408181487e-06
epoch > 50000 train loss = 1207.13427734375 val loss = 1115.6507568359375 lr = 3.333355555703705e-06
epoch > 60000 train loss = 1090.8802490234375 val loss = 1008.2244262695312 lr = 3.125019531372071e-06
epoch > 70000 train loss = 974.9158935546875 val loss = 886.576416015625 lr = 2.941193771728069e-06
epoch > 80000 train loss = 935.328369140625 val loss = 851.0785522460938 lr = 2.7777932099622774e-06
epoch > 90000 train loss = 912.2388916015625 val loss = 834.6104736328125 lr = 2.6315927978568314e-06
epoch > 10

In [23]:
import time

In [48]:
kf = KFold(10,shuffle=True,random_state=40)

In [49]:
summary = []
i = 1
for train_index, test_index in kf.split(data):
  train,test = data.iloc[train_index],data.iloc[test_index]
  model = Model(1,device='cpu',loss=mae)
  start_time = time.perf_counter()
  model.fit(train,alpha=5e-6,n_epoch=int(2e6),_print=False)
  end_time = time.perf_counter()
  intersect, base_week, current_week, features, target_value = model.get_data(test)
  p = model.predict(intersect,base_week,current_week,features)
  print(f'model->{i}',mae(target_value,p).item(),'time(sec) -> ',end_time-start_time)
  summary.append((mae(target_value,p).item(),end_time-start_time))
  i+=1
  torch.save(model.W,f'final_model{i}.pt')

model->1 169.0091094970703 time(sec) ->  548.3050181810004
model->2 148.1373291015625 time(sec) ->  550.598870923


KeyboardInterrupt: ignored